# 쿠팡 데이터 탐색적 데이터 분석 (EDA)

쿠팡 상품, 판매자, 리뷰 및 질문 데이터를 분석하여 데이터의 분포와 특징을 파악합니다.

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

## 1. 데이터 로드

Supabase에서 주요 데이터들을 불러옵니다.

In [2]:
import sys
sys.path.append(os.path.abspath('..'))

from src.database import load_table

products_df = load_table('products')
sellers_df = load_table('sellers')
reviews_df = load_table('reviews')
questions_df = load_table('questions')

print(f"상품 데이터: {products_df.shape}")
print(f"판매자 데이터: {sellers_df.shape}")
print(f"리뷰 데이터: {reviews_df.shape}")
print(f"질문 데이터: {questions_df.shape}")

상품 데이터: (416, 17)
판매자 데이터: (403, 14)
리뷰 데이터: (355, 7)
질문 데이터: (1000, 7)


## 2. 데이터 기본 정보 확인

각 데이터프레임의 컬럼 정보와 누락된 값(Null)을 확인합니다.

In [3]:
print("--- 상품 데이터 정보 ---")
print(products_df.info())
print("\n--- 결측치 분포 ---")
print(products_df.isnull().sum())

--- 상품 데이터 정보 ---
<class 'pandas.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                416 non-null    int64  
 1   product_id        416 non-null    str    
 2   name              416 non-null    str    
 3   item_id           416 non-null    str    
 4   review_count      416 non-null    int64  
 5   vendor_name       416 non-null    str    
 6   price             416 non-null    int64  
 7   original_price    416 non-null    int64  
 8   candidate_price   416 non-null    int64  
 9   discount_rate     416 non-null    int64  
 10  is_seller_rocket  416 non-null    bool   
 11  product_rating    416 non-null    float64
 12  is_overseas       0 non-null      object 
 13  category          416 non-null    str    
 14  shipping_fee      416 non-null    int64  
 15  shipping_days     416 non-null    int64  
 16  created_at        416 non-null    str

## 3. 주요 지표 분포 분석

### 3.1 가격 분포

In [4]:
fig = go.Figure(data=[go.Histogram(x=products_df['price'], nbinsx=50, marker_color='#636EFA')])
fig.update_layout(
    title='상품 가격 분포',
    xaxis_title='가격',
    yaxis_title='빈도',
    template='plotly_white'
)
fig.show()

### 3.2 평점 및 리뷰 수 분포

In [5]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('상품 평점 분포', '리뷰 수 분포'))

fig.add_trace(
    go.Histogram(x=products_df['product_rating'], nbinsx=10, name='평점', marker_color='#EF553B'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=products_df['review_count'], nbinsx=50, name='리뷰 수', marker_color='#00CC96'),
    row=1, col=2
)

fig.update_layout(height=500, title_text="평점 및 리뷰 수 종합 분석", template='plotly_white')
fig.show()

## 4. 판매자 분석

판매자의 만족도 점수와 등록된 상품 수를 분석합니다.

In [6]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=sellers_df['satisfaction_score'], nbinsx=20, marker_color='#AB63FA'))
fig.update_layout(
    title='판매자 만족도 점수 분포',
    xaxis_title='만족도',
    yaxis_title='빈도',
    template='plotly_white'
)
fig.show()

## 5. 상관관계 분석

수치형 변수들 간의 상관관계를 확인합니다.

In [7]:
numeric_df = products_df.select_dtypes(include=['float64', 'int64'])
corr = numeric_df.corr()

# 컬럼명 한글 매핑
column_mapping = {
    'product_id': '상품ID',
    'item_id': '아이템ID',
    'review_count': '리뷰 수',
    'price': '가격',
    'original_price': '정가',
    'candidate_price': '후보가격',
    'discount_rate': '할인율',
    'product_rating': '평점',
    'shipping_fee': '배송비',
    'shipping_days': '배송일수'
}

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=[column_mapping.get(col, col) for col in corr.columns],
    y=[column_mapping.get(col, col) for col in corr.index],
    colorscale='RdBu_r',
    zmin=-1, zmax=1,
    text=corr.values.round(2),
    texttemplate='%{text}',
    textfont=dict(size=10)
))

fig.update_layout(
    title='상태 지표 간 상관관계 히트맵',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    template='plotly_white'
)
fig.show()

## 6. 어뷰징 판매자 분석

정상 판매자와 어뷰징 판매자의 특성을 비교 분석합니다.

In [8]:
# 어뷰징 vs 정상 판매자 비교
abusing_sellers = sellers_df[sellers_df['is_abusing_seller']]
normal_sellers = sellers_df[~sellers_df['is_abusing_seller']]

print(f"어뷰징 판매자: {len(abusing_sellers)}개 ({len(abusing_sellers)/len(sellers_df)*100:.1f}%)")
print(f"정상 판매자: {len(normal_sellers)}개 ({len(normal_sellers)/len(sellers_df)*100:.1f}%)")

# 주요 지표 비교
fig = make_subplots(rows=1, cols=3, subplot_titles=('만족도 점수', '리뷰 수', '등록 상품 수'))

for i, (col, title) in enumerate([('satisfaction_score', '만족도'), ('review_count', '리뷰 수'), ('total_product_count', '상품 수')], 1):
    fig.add_trace(go.Box(y=normal_sellers[col], name='정상', marker_color='#00CC96'), row=1, col=i)
    fig.add_trace(go.Box(y=abusing_sellers[col], name='어뷰징', marker_color='#EF553B'), row=1, col=i)

fig.update_layout(height=400, title_text="어뷰징 vs 정상 판매자 지표 비교", showlegend=False, template='plotly_white')
fig.show()

어뷰징 판매자: 254개 (63.0%)
정상 판매자: 149개 (37.0%)


## 7. 카테고리 분석

상품의 카테고리 분포와 카테고리별 특성을 분석합니다.

In [9]:
# 1차 카테고리 추출 (첫 번째 > 이전 부분)
products_df['category_1st'] = products_df['category'].apply(
    lambda x: x.split(' > ')[0] if pd.notna(x) and ' > ' in str(x) else x
)

# 카테고리별 상품 수
category_counts = products_df['category_1st'].value_counts().head(15)

fig = go.Figure(data=[
    go.Bar(x=category_counts.values, y=category_counts.index, orientation='h', marker_color='#636EFA')
])
fig.update_layout(
    title='1차 카테고리별 상품 수 (Top 15)',
    xaxis_title='상품 수',
    yaxis_title='카테고리',
    height=500,
    template='plotly_white',
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

In [10]:
# 카테고리별 평균 가격 및 평점
category_stats = products_df.groupby('category_1st').agg({
    'price': 'mean',
    'product_rating': 'mean',
    'review_count': 'sum'
}).round(2)

top_categories = category_counts.head(10).index.tolist()
category_stats_top = category_stats.loc[category_stats.index.isin(top_categories)]

fig = make_subplots(rows=1, cols=2, subplot_titles=('카테고리별 평균 가격', '카테고리별 평균 평점'))

fig.add_trace(
    go.Bar(x=category_stats_top['price'], y=category_stats_top.index, orientation='h', 
           name='평균 가격', marker_color='#636EFA'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=category_stats_top['product_rating'], y=category_stats_top.index, orientation='h',
           name='평균 평점', marker_color='#EF553B'),
    row=1, col=2
)

fig.update_layout(height=500, title_text="카테고리별 가격 및 평점 분석", template='plotly_white', showlegend=False)
fig.show()

## 8. 리뷰 심층 분석

리뷰 평점 분포, 텍스트 길이, 시간대별 분포를 분석합니다.

In [11]:
# 리뷰 평점 분포
rating_counts = reviews_df['review_rating'].value_counts().sort_index()

fig = go.Figure(data=[
    go.Bar(x=rating_counts.index.astype(str), y=rating_counts.values, 
           marker_color=['#EF553B', '#FFA15A', '#FECB52', '#00CC96', '#636EFA'])
])
fig.update_layout(
    title='리뷰 평점 분포',
    xaxis_title='평점',
    yaxis_title='리뷰 수',
    template='plotly_white'
)
fig.show()

# 평점별 통계
print("\n=== 평점별 리뷰 수 ===")
for rating in sorted(reviews_df['review_rating'].unique()):
    count = len(reviews_df[reviews_df['review_rating'] == rating])
    pct = count / len(reviews_df) * 100
    print(f"  {rating}점: {count}개 ({pct:.1f}%)")


=== 평점별 리뷰 수 ===
  1점: 67개 (18.9%)
  2점: 22개 (6.2%)
  3점: 47개 (13.2%)
  4점: 217개 (61.1%)
  5점: 2개 (0.6%)


In [12]:
# 리뷰 텍스트 길이 분석
reviews_df['text_length'] = reviews_df['review_text'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

fig = make_subplots(rows=1, cols=2, subplot_titles=('리뷰 텍스트 길이 분포', '평점별 평균 텍스트 길이'))

fig.add_trace(
    go.Histogram(x=reviews_df['text_length'], nbinsx=30, marker_color='#636EFA'),
    row=1, col=1
)

avg_length_by_rating = reviews_df.groupby('review_rating')['text_length'].mean()
fig.add_trace(
    go.Bar(x=avg_length_by_rating.index.astype(str), y=avg_length_by_rating.values, marker_color='#EF553B'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="리뷰 텍스트 길이 분석", template='plotly_white', showlegend=False)
fig.show()

print(f"\n평균 리뷰 길이: {reviews_df['text_length'].mean():.1f}자")
print(f"최대 리뷰 길이: {reviews_df['text_length'].max()}자")


평균 리뷰 길이: 443.3자
최대 리뷰 길이: 3679자


## 9. 질문-답변 분석

상품 문의의 답변율과 답변 시간을 분석합니다.

In [13]:
# 답변 여부 분석
questions_df['has_answer'] = questions_df['answer'].apply(lambda x: pd.notna(x) and str(x).strip() != '')

answered = questions_df['has_answer'].sum()
unanswered = len(questions_df) - answered

print(f"총 질문 수: {len(questions_df)}개")
print(f"답변 완료: {answered}개 ({answered/len(questions_df)*100:.1f}%)")
print(f"미답변: {unanswered}개 ({unanswered/len(questions_df)*100:.1f}%)")

# 파이 차트
fig = go.Figure(data=[go.Pie(
    labels=['답변 완료', '미답변'],
    values=[answered, unanswered],
    marker_colors=['#00CC96', '#EF553B'],
    hole=0.4
)])
fig.update_layout(title='질문 답변율', template='plotly_white')
fig.show()

총 질문 수: 1000개
답변 완료: 975개 (97.5%)
미답변: 25개 (2.5%)


## 10. 가격 이상치 및 할인율 분석

비정상적인 가격이나 할인율을 가진 상품을 분석합니다.

In [14]:
# 가격 통계
print("=== 가격 통계 ===")
print(products_df['price'].describe())

# 가격 범위별 분포
price_bins = [0, 10000, 50000, 100000, 500000, float('inf')]
price_labels = ['~1만원', '1~5만원', '5~10만원', '10~50만원', '50만원+']
products_df['price_range'] = pd.cut(products_df['price'], bins=price_bins, labels=price_labels)

price_range_counts = products_df['price_range'].value_counts()

fig = go.Figure(data=[go.Pie(
    labels=price_range_counts.index.tolist(),
    values=price_range_counts.values,
    hole=0.3
)])
fig.update_layout(title='가격대별 상품 분포', template='plotly_white')
fig.show()

# 할인율 분포
fig = go.Figure(data=[go.Histogram(x=products_df['discount_rate'], nbinsx=20, marker_color='#FFA15A')])
fig.update_layout(
    title='할인율 분포',
    xaxis_title='할인율 (%)',
    yaxis_title='상품 수',
    template='plotly_white'
)
fig.show()

=== 가격 통계 ===
count    4.160000e+02
mean     4.574358e+04
std      9.203253e+04
min      2.830000e+03
25%      1.468750e+04
50%      2.301500e+04
75%      3.649250e+04
max      1.190000e+06
Name: price, dtype: float64


## 11. 판매자별 상품 분포

판매자당 등록된 상품 수 분포를 분석합니다.

In [15]:
# 판매자별 상품 수 (데이터 내)
vendor_product_counts = products_df['vendor_name'].value_counts()

print(f"총 판매자 수: {len(vendor_product_counts)}개")
print(f"판매자당 평균 상품 수: {vendor_product_counts.mean():.2f}개")
print("\n=== 상품이 많은 판매자 Top 10 ===")
print(vendor_product_counts.head(10))

# 분포 시각화
fig = go.Figure(data=[go.Histogram(x=vendor_product_counts.values, nbinsx=20, marker_color='#AB63FA')])
fig.update_layout(
    title='판매자별 상품 수 분포',
    xaxis_title='상품 수',
    yaxis_title='판매자 수',
    template='plotly_white'
)
fig.show()

총 판매자 수: 378개
판매자당 평균 상품 수: 1.10개

=== 상품이 많은 판매자 Top 10 ===
vendor_name
노빈라이트 유한회사    4
아이와이2상회       4
유니마트          3
선스마켓          3
Coupang       3
원가상점          3
케이와이컴퍼니       2
하오란 유한회사      2
패스트몰          2
기가철물          2
Name: count, dtype: int64


## 12. 데이터 요약

전체 데이터의 주요 통계를 요약합니다.

In [16]:
# 종합 요약 테이블
summary_data = {
    '지표': [
        '총 상품 수',
        '총 판매자 수',
        '총 리뷰 수',
        '총 질문 수',
        '어뷰징 판매자 비율',
        '평균 상품 가격',
        '평균 상품 평점',
        '평균 리뷰 길이',
        '질문 답변율',
        '주요 카테고리'
    ],
    '값': [
        f"{len(products_df):,}개",
        f"{len(sellers_df):,}개",
        f"{len(reviews_df):,}개",
        f"{len(questions_df):,}개",
        f"{len(abusing_sellers)/len(sellers_df)*100:.1f}%",
        f"{products_df['price'].mean():,.0f}원",
        f"{products_df['product_rating'].mean():.2f}점",
        f"{reviews_df['text_length'].mean():.0f}자",
        f"{answered/len(questions_df)*100:.1f}%",
        category_counts.index[0]
    ]
}

summary_df = pd.DataFrame(summary_data)
print("=" * 50)
print("데이터 종합 요약")
print("=" * 50)
print(summary_df.to_string(index=False))

데이터 종합 요약
        지표       값
    총 상품 수    416개
   총 판매자 수    403개
    총 리뷰 수    355개
    총 질문 수  1,000개
어뷰징 판매자 비율   63.0%
  평균 상품 가격 45,744원
  평균 상품 평점   4.74점
  평균 리뷰 길이    443자
    질문 답변율   97.5%
   주요 카테고리      식품
